# Data Exploration
* Data from: https://open.fda.gov/apis/drug/event/download/
* API: https://api.fda.gov/drug/event.json

In [26]:
# import libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.sparse import coo_matrix

### Load all data

In [2]:
filenames = [f'./data/data_2018_q{i}.csv' for i in [1, 2, 3, 4]]
df_list = []
for f in filenames:
    print(f'reading file {f}')
    df_list.append(pd.read_csv(f))
df_original = pd.concat(df_list, ignore_index=True, sort=True)
del df_list
df_original.head(20)

reading file ./data/data_2018_q1.csv
reading file ./data/data_2018_q2.csv
reading file ./data/data_2018_q3.csv
reading file ./data/data_2018_q4.csv


,Unnamed: 0,brand_name,generic_name,manufacturer_name,occurcountry,patientonsetage,patientonsetageunit,patientsex,product_type,reactionmeddrapt,reactionoutcome,receivedate,route,safetyreportid,serious,substance_name
0,0,NaN,NaN,NaN,SE,66.0,801.0,1.0,NaN,Hepatic failure,3.0,20171010,NaN,14068265,1,NaN
1,1,NaN,NaN,NaN,SE,66.0,801.0,1.0,NaN,Cardiac failure,3.0,20171010,NaN,14068265,1,NaN
2,2,NaN,NaN,NaN,SE,66.0,801.0,1.0,NaN,Multiple organ dysfunction syndrome,3.0,20171010,NaN,14068265,1,NaN
3,3,NaN,NaN,NaN,SE,66.0,801.0,1.0,NaN,Tubulointerstitial nephritis,3.0,20171010,NaN,14068265,1,NaN
4,4,JEVTANA,CABAZITAXEL,Sanofi-Aventis U.S. LLC,JP,NaN,NaN,1.0,HUMAN PRESCRIPTION DRUG,Death,5.0,20180227,NaN,14578282,1,NaN
5,5,ONZETRA,SUMATRIPTAN SUCCINATE,"Avanir Pharmaceuticals, Inc.",US,NaN,NaN,1.0,HUMAN PRESCRIPTION DRUG,Drug ineffective,6.0,20171107,NASAL,14167252,2,NaN
6,6,ONZETRA,SUMATRIPTAN SUCCINATE,"Avanir Pharmaceuticals, Inc.",US,NaN,NaN,1.0,HUMAN PRESCRIPTION DRUG,No adverse event,6.0,20171107,NASAL,14167252,2,NaN
7,7,NaN,NaN,NaN,US,62.0,801.0,1.0,NaN,Heart rate increased,6.0,20180308,NaN,14613893,2,NaN
8,8,NaN,NaN,NaN,US,62.0,801.0,1.0,NaN,Heart rate increased,6.0,20180308,NaN,14613893,2,NaN
9,9,NaN,NaN,NaN,US,62.0,801.0,1.0,NaN,Chest discomfort,6.0,20180308,NaN,14613893,2,NaN


### Basic exporation

#### Missing data stat

In [3]:
df_original.isna().sum()

Unnamed: 0                   0
brand_name             1328120
generic_name           1328068
manufacturer_name      1328068
occurcountry               303
patientonsetage        3892682
patientonsetageunit    3892621
patientsex             1215075
product_type           1328068
reactionmeddrapt             0
reactionoutcome         550983
receivedate                  0
route                  1758827
safetyreportid               0
serious                      0
substance_name         1792762
dtype: int64

In [4]:
df_astra = df_original.loc[df_original['manufacturer_name'] == 'AstraZeneca Pharmaceuticals LP', :]
len(df_astra)

279362

#### Different countries report different events?
To answer this question, we could compute the record counts of each country and each adverse event type (MedDRA PT), and then calculate the cosine similarity between each country and United States (where we have most of the records).

Eventually we can get a ranking of similarities from 0 to 1.

Of course, we could compute the similarity "coefficient" among each pair of countries..

In [5]:
def compute_correlation(df_original):
    cols = ['occurcountry', 'reactionmeddrapt']
    df_proj = df_original[cols].copy()
    df_proj['count'] = 1
    df_count = df_proj.groupby(cols).count().reset_index()
    country_reaction_mat_df = pd.pivot_table(df_count, index='occurcountry', columns='reactionmeddrapt', fill_value=0)

    us_row = country_reaction_mat_df.loc['US', :].values  # as a reference
    matrix = country_reaction_mat_df.values
    # normalize the vectors
    us_row = us_row / np.linalg.norm(us_row)
    matrix = matrix / (np.sqrt(np.sum(np.square(matrix), axis=1))).reshape((-1, 1))
    correlation = np.matmul(matrix, us_row.reshape((-1, 1)))
    correlation.shape
    country_reaction_mat_df['corr_us'] = correlation
    relation_df = country_reaction_mat_df['corr_us'].sort_values(ascending=False)
    return relation_df

relation_df = compute_correlation(df_original)
relation_df_Nexium = compute_correlation(df_astra.loc[df_astra['brand_name'] == 'NEXIUM', :])

In [6]:
pd.DataFrame(relation_df).head(10)

,corr_us
occurcountry,
US,1.000000
BR,0.817468
AU,0.814663
GB,0.800400
MX,0.799618
CA,0.794560
NL,0.787933
CO,0.769467
IE,0.755631


Brazil, Australia, UK, Mexico, and Canada have pretty similar adverse event reporting types as US.

Now let's check the controversial Nexium

In [7]:
pd.DataFrame(relation_df_Nexium).head(10)

,corr_us
occurcountry,
US,1.000000
DO,0.373476
MY,0.193817
CL,0.181544
PT,0.148828
IE,0.130488
CH,0.129576
AU,0.120243
JP,0.108130


The most similar country is.. Dominican republic and then Malaysia, and the score is quite low!

Only US people are reporting kidney problems -- not sure what's going on

#### What drugs are taken together?

In [15]:
def drugs_together(df_original):
    cols = ['safetyreportid', 'brand_name']
    df_proj = df_original[cols].copy().dropna().drop_duplicates().reset_index(drop=True)
    df_nummeds = df_proj.groupby('safetyreportid').count()
    df_nummeds = df_nummeds.loc[df_nummeds['brand_name'] > 1, :]
    df_multiple = pd.DataFrame(df_nummeds.reset_index()['safetyreportid']).merge(df_original[['safetyreportid', 'brand_name', 'manufacturer_name']], on='safetyreportid').dropna()
    df_pairs = df_multiple[['safetyreportid', 'brand_name']].drop_duplicates().reset_index(drop=True)
    return df_pairs

df_pairs = drugs_together(df_original)

In [30]:
id_le = preprocessing.LabelEncoder()
bn_le = preprocessing.LabelEncoder()
id_x = df_pairs['safetyreportid'].values
bn_y = df_pairs['brand_name'].values
id_le.fit(id_x)
bn_le.fit(bn_y)
id_idx = id_le.transform(id_x)
bn_idx = bn_le.transform(bn_y)

In [32]:
ones = np.ones(len(id_idx))
corr_matrix = coo_matrix((ones, (id_idx, bn_idx)))

In [33]:
corr_matrix

<464716x3818 sparse matrix of type '<class 'numpy.float64'>'
	with 2094349 stored elements in COOrdinate format>

In [37]:
len(bn_le.classes_)

3818